In [ ]:
import numpy as np, os, time, matplotlib.pyplot as plt

# Add a legend and grid to a `matplotlib.pyplot` plot, and resize it

In [ ]:
def plot_util(_plt, height_inch=5):
    _plt.legend()
    _plt.grid()
    fig = _plt.gcf()
    fig.set_size_inches(18.5, height_inch, forward=True)
    _plt.show()

# Convert the output from an `SSD` neural network into an `output_dict`

In [ ]:
# ****************************************************************************
# Copyright(c) 2017 Intel Corporation. 
# License: MIT See LICENSE file in root directory.
# ****************************************************************************

# Utilities to help deserialize the output list from
# Intel® Movidius™ Neural Compute Stick (NCS)
def deserialize_ssd(output, shape, confidance_threshold):
    """---- Deserialize the output from an SSD based network ----
    
    @param output The NCS returns a list/array in this structure:
        First float16: Number of detections
        Next 6 values: Unused
        Next consecutive batch of 7 values: Detection values
          0: Image ID (always 0)
          1: Class ID (index into labels.txt)
          2: Detection score
          3: Box left coordinate (x1) - scaled value between 0 & 1
          4: Box top coordinate (y1) - scaled value between 0 & 1
          5: Box right coordinate (x2) - scaled value between 0 & 1
          6: Box bottom coordinate (y2) - scaled value between 0 & 1

    @return output_dict A Python dictionary with the following keys:
        output_dict['num_detections'] = Total number of valid detections
        output_dict['detection_classes_<X>'] = Class ID of the detected object
        output_dict['detection_scores_<X>'] = Percentage of the confidance
        output_dict['detection_boxes_<X>'] = A list of 2 tuples [(x1, y1) (x2, y2)]
        Where <X> is a zero-index count of num_detections
    """

    output_dict = {}                # Dictionary where the deserialized output will be stored
    height, width = shape           # Extract the original image's shape
    channel = 3
    output_dict['num_detections'] = int(output[0])  # Total number of detections
    num_valid_detections = 0

    for detection in range(output_dict['num_detections']):
        base_index = 7 + (7 * detection)  # Skip the first 7 values

        if (output[base_index + 2] > confidance_threshold):
            output_dict['detection_classes_' + str(num_valid_detections)] = int(output[base_index + 1])
            output_dict['detection_scores_' + str(num_valid_detections)] = int(output[base_index + 2] * 100)

            x = [int(output[base_index + 3] * width), int(output[base_index + 5] * width)]
            y = [int(output[base_index + 4] * height), int(output[base_index + 6] * height)]

            output_dict['detection_boxes_' + str(num_valid_detections)] = list(zip(y, x))

            num_valid_detections += 1

    # Update total number of detections to valid detections
    output_dict['num_detections'] = int(num_valid_detections)

    return output_dict

# Convert the `output_dict` to tuples of bounding boxes (pixels and angles angles)

In [ ]:
def output_dict_to_bb_and_angles(output_dict, class_of_interest, camera_resolution, camera_FOV):
    for i in range(output_dict['num_detections']):
        if (output_dict.get('detection_classes_%i' % i) == class_of_interest):
            (y1, x1) = output_dict.get('detection_boxes_' + str(i))[0]
            (y2, x2) = output_dict.get('detection_boxes_' + str(i))[1]
            bb = (x1, y1), (x2, y2)

            w, h = camera_resolution
            cam_width = camera_FOV[0]
            cam_height = camera_FOV[1]
            x1_angle = pixel_to_angle(x1, w, cam_width)
            x2_angle = pixel_to_angle(x2, w, cam_width)
            y1_angle = pixel_to_angle(y1, h, cam_height)
            y2_angle = pixel_to_angle(y2, h, cam_height)
            bb_angles = ((x1_angle, y1_angle), (x2_angle, y2_angle))

            return bb, bb_angles
    return -1, -1

def pixel_to_angle(pixel, img_size, cam_angle_deg):
    """ convert a pixel value to an angle in degrees
    inputs:
        pixel:              a value from 0 to img_size
        img_size:           the total number of pixels along that axis of the image
        cam_angle_deg:      the angular width of camera in degrees
        
    output:
        angle:      0deg = when pixel is directly ahead, positive/negative = right/left of center
    """
    norm_pixel = (2*pixel/img_size) - 1  # now in range [-1, 1]
    angle = np.arctan(norm_pixel * np.tan(np.deg2rad(cam_angle_deg/2)))
    return np.rad2deg(angle)

# Overlay data from `output_dict` onto an image

In [ ]:
import PIL.Image, PIL.ImageDraw, PIL.ImageFont

def display_image(output_dict, class_of_interest, frame, labels, capture_screenshots=False):
    # Print the results (each image/frame may have multiple objects)
    for i in range(output_dict['num_detections']):
        
        if class_of_interest =='all' or (output_dict.get('detection_classes_' + str(i)) == class_of_interest):
            
            # Extract top-left & bottom-right coordinates of detected objects
            (y1, x1) = output_dict.get('detection_boxes_' + str(i))[0]
            (y2, x2) = output_dict.get('detection_boxes_' + str(i))[1]

            # Prep string to overlay on the image
            display_str = (labels[output_dict.get('detection_classes_%i' % i)]
                           + ': %s%%' % output_dict.get('detection_scores_%i' % i))

            # Overlay bounding boxes, detection class and scores
            frame = draw_bounding_box( 
                        y1, x1, y2, x2,
                        frame, display_str=display_str)

    if capture_screenshots:
        img = PIL.Image.fromarray(frame)
        img.save('captures/photo_%s.jpg' % cur_time)

    # If a display is available, show image on which inference was performed
    if 'DISPLAY' in os.environ:
        img.show()
    
    return frame

# ****************************************************************************
# Copyright(c) 2017 Intel Corporation. 
# License: MIT See LICENSE file in root directory.
# ****************************************************************************

# Utilities to help visualize the output from
# Intel® Movidius™ Neural Compute Stick (NCS)
def draw_bounding_box(y1, x1, y2, x2, 
                      img, 
                      thickness=4, 
                      color=(255, 255, 0),
                      display_str=()):
    """ draw a bounding box on an image to help visualise the nn output
    
    Inputs
        (x1, y1)  = Top left corner of the bounding box
        (x2, y2)  = Bottom right corner of the bounding box
        img       = Image/frame represented as numpy array
        thickness = Thickness of the bounding box's outline
        color     = Color of the bounding box's outline
    """
    img = PIL.Image.fromarray(img)
    draw = PIL.ImageDraw.Draw(img)

    for x in range(0, thickness):
        draw.rectangle([(x1-x, y1-x), (x2-x, y2-x)], outline=color)

    font = PIL.ImageFont.load_default()
    draw.text((x1, y1), display_str, font=font)

    return np.array(img)

# Logger to keep track of `EKF` angles, and plot afterwards

In [ ]:
class ExperimentLogger():
    """ Keeps track of,
            obj_xxx_arr --> estimate of relative angle to object, from nn 
            EKF_xxx_arr --> estimate of absolute angle of object, from EKF
            gc_xxx_arr ---> estimate of absolute angle of camera, from gimbal
         
        where 'xxx' is 'pitch' or 'yaw'
        
        Also provides a convenience function to plot all of this afterwards. 
        """
    def __init__(self):
        self.obj_yaw_arr = []
        self.EKF_yaw_arr = []
        self.gc_yaw_arr = []

        self.obj_pitch_arr = []
        self.EKF_pitch_arr = []
        self.gc_pitch_arr = []

        self.time_arr = []
    
    def log(self, gc_angles, obj_yaw, EKF_yaw, obj_pitch, EKF_pitch, t):
        self.obj_yaw_arr.append(obj_yaw)
        self.EKF_yaw_arr.append(EKF_yaw.get_cur_est_pos())

        self.obj_pitch_arr.append(obj_pitch)
        self.EKF_pitch_arr.append(EKF_pitch.get_cur_est_pos())

        self.gc_yaw_arr.append(gc_angles['yaw'])
        self.gc_pitch_arr.append(gc_angles['pitch'])

        self.time_arr.append(t)
    
    def plot(self):
        plt.subplot(211)
        plt.plot(self.time_arr, self.EKF_yaw_arr, label='EKF estimate of yaw [absolute][deg]')
        plt.plot(self.time_arr, self.obj_yaw_arr, label='Raw NN estimate of yaw [relative][deg]')
        plt.plot(self.time_arr, self.gc_yaw_arr, label='Gimbal yaw [absolute][deg]')
        plot_util(plt)

        plt.subplot(212)
        plt.plot(self.time_arr, self.EKF_pitch_arr, label='EKF estimate of pitch [absolute][deg]')
        plt.plot(self.time_arr, self.obj_pitch_arr, label='Raw NN estimate of pitch [relative][deg]')
        plt.plot(self.time_arr, self.gc_pitch_arr, label='Gimbal pitch [absolute][deg]')
        plot_util(plt)

        loop_times_ms = [(t-t_)*1e3 for t_,t in zip(self.time_arr[0:-1], self.time_arr[1:])]
        plt.stem(self.time_arr[:-1], loop_times_ms, label='loop times [ms]')
        plot_util(plt)

# Rolling buffer for gimbal angles

In [ ]:
class GimbalAngleBuffer():
    """ Keeps a rolling buffer of the last `buffer_len` angle readings from the gimbal,
        including the time at which the angle readings were taken.
        
        If given the time at which a photo was taken, it returns the gimbal angle log
        which was taken close to that time. """
    def __init__(self, buffer_len=20):
        self.buffer_len = buffer_len
        self.time_buffer = np.zeros(buffer_len)
        self.yaw_buffer = np.zeros(buffer_len)
        self.pitch_buffer = np.zeros(buffer_len)
        self.idx = 0
    
    def log(self, gc_angles): # gc_angles is the dict returned from gc.get_motor_angles()
        self.time_buffer[self.idx % self.buffer_len] = time.time()
        self.yaw_buffer[self.idx % self.buffer_len] = gc_angles['yaw']
        self.pitch_buffer[self.idx % self.buffer_len] = gc_angles['pitch']
        self.idx += 1

    def angle_closest_to(self, photo_time):
        idx = np.argmin(np.abs(self.time_buffer - photo_time)) # time closest to the photo time
        return self.yaw_buffer[idx], self.pitch_buffer[idx]

In [ ]:
if __name__ == '__main__': # test/understand the rolling buffer
    import time

    x = GimbalAngleBuffer(4)

    gc_angles1 = {'yaw': 20, 'pitch': 15}
    gc_angles2 = {'yaw': 30, 'pitch': 25}
    gc_angles3 = {'yaw': 40, 'pitch': 35}
    gc_angles4 = {'yaw': 50, 'pitch': 45}

    x.log(gc_angles1)
    time.sleep(0.5)

    x.log(gc_angles2)
    time.sleep(0.2)
    photo_time = time.time()
    time.sleep(0.5)

    x.log(gc_angles3)
    time.sleep(0.5)

    x.log(gc_angles4)
    time.sleep(0.5)

    print(x.angle_closest_to(photo_time)) # should be (30, 25), as it corresponds to the angle nearest to the photo_time